# Práctica de Bioinformática - Sesión 04
## Uso de BLAST con Biopython

Autores: **Raúl Mendoza**, **Adrián Ojeda**  
Asignatura: **Bioinformática (ULPGC)**

En esta práctica trabajamos con BLAST (Basic Local Alignment Search Tool) para buscar secuencias similares en bases de datos biológicas, usando **Biopython** tanto en modo online (servidores de NCBI) como en modo local (BLAST+ instalado en nuestra máquina).

> Nota importante
>
> En este cuaderno programamos todas las soluciones, pero muchas celdas que hacen consultas reales a BLAST NO se ejecutan aquí porque necesitan conexión a internet y/o > tener BLAST+ instalado con sus bases de datos. La idea es que el código se pueda > copiar tal cual en nuestro propio entorno y ejecutar desde ahí.

## Requisitos previos

- Python 3.x
- Paquete **Biopython** instalado:
  ```bash
  pip install biopython
  ```
- Para la parte **online**: conexión a internet y una dirección de correo (NCBI la pide por cortesía para identificar al usuario).
- Para la parte **local**:
  - Tener **BLAST+** instalado (ejecutables como `blastn`, `blastp`, `blastx`, `tblastn`,     `tblastx`).
  - Tener al menos una base de datos BLAST ya creada (por ejemplo con `makeblastdb`).
  - Tener el directorio de los ejecutables en el `PATH` o indicar la ruta completa.

In [45]:
# Imports generales que usaremos en toda la práctica
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast import Applications
from Bio.Blast.Applications import (
    NcbiblastnCommandline,
    NcbiblastpCommandline,
    NcbiblastxCommandline,
    NcbitblastnCommandline,
    NcbitblastxCommandline,
)

from Bio import SeqIO
import os
import tempfile
import statistics


### Funciones auxiliares comunes

Antes de meternos con los ejercicios, defino algunas funciones que reutilizo varias veces: escribir una secuencia a FASTA temporal, calcular porcentaje de identidad y cobertura, etc.

In [60]:
def escribir_fasta_temporal(seq, descripcion="query"):
    """Guarda la secuencia en un fichero FASTA temporal y devuelve la ruta.

    Esto es útil para llamar a BLAST local, que normalmente espera un archivo.
    """
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".fasta", mode="w")
    tmp.write(f">{descripcion}\n")
    seq_limpia = ''.join(seq.split()).upper()
    tmp.write(seq_limpia + "\n")
    tmp.close()
    return tmp.name


def porcentaje_identidad(hsp):
    """Devuelve el porcentaje de identidad de un HSP."""
    if hsp.align_length == 0:
        return 0.0
    return 100.0 * hsp.identities / hsp.align_length


def cobertura_query(hsp, query_length):
    """Porcentaje de la query cubierta por este HSP."""
    if query_length == 0:
        return 0.0
    return 100.0 * hsp.align_length / query_length


## Ejercicio 1

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de ADN que introduzcas por teclado. El programa debe mostrar por pantalla:
- el número de resultados obtenidos,
- el E-value del mejor resultado,
- y la descripción de la secuencia más similar.
Hágalo de forma online y local.


### 1.a) Búsqueda BLASTN online

Usamos `NCBIWWW.qblast` para enviar la secuencia al servidor BLAST de NCBI y parseamos el XML devuelto con `NCBIXML`.

In [61]:
def blastn_online_desde_teclado():
    from Bio.Blast import NCBIWWW, NCBIXML

    secuencia = input("Introduce la secuencia de ADN (sin espacios): ")
    secuencia = ''.join(secuencia.split()).upper()

    print("\nLanzando BLASTN online... (puede tardar)")
    result_handle = NCBIWWW.qblast(
        program="blastn",
        database="nt",
        sequence=secuencia,
        expect=0.05,
        hitlist_size=50,
        megablast=True,
    )

    record = NCBIXML.read(result_handle)
    result_handle.close()

    num_hits = len(record.alignments)
    print(f"Número de hits: {num_hits}")

    if num_hits == 0:
        print("No se encontraron resultados.")
        return

    best_alignment = record.alignments[0]
    best_hsp = best_alignment.hsps[0]

    print(f"Mejor e-value: {best_hsp.expect}")
    print("Descripción del mejor hit:")
    print(best_alignment.hit_def)

# Para usar en local, descomentar:
# blastn_online_desde_teclado()



Lanzando BLASTN online... (puede tardar)


KeyboardInterrupt: 

### 1.b) Búsqueda BLASTN local

Ahora hacemos lo mismo, pero usando BLAST+ instalado en local. Asumimos que:
- `blastn` está en el PATH,
- existe una BD local de nucleótidos, por ejemplo `genomasbase`.

In [48]:
def blastn_local_desde_teclado(
    db="genomasbase",
    blastn_path="blastn",
    out_xml="blastn_local_result.xml",
):
    from Bio.Blast import NCBIXML

    secuencia = input("Introduce la secuencia de ADN (sin espacios): ")
    secuencia = ''.join(secuencia.split()).upper()

    query_fasta = escribir_fasta_temporal(secuencia, descripcion="query_local")

    blastn_cline = NcbiblastnCommandline(
        cmd=blastn_path,
        query=query_fasta,
        db=db,
        evalue=0.05,
        outfmt=5,
        out=out_xml,
    )

    print("Ejecutando BLASTN local...")
    stdout, stderr = blastn_cline()

    with open(out_xml) as handle:
        record = NCBIXML.read(handle)

    num_hits = len(record.alignments)
    print(f"Número de hits: {num_hits}")

    if num_hits == 0:
        print("No se encontraron resultados.")
    else:
        best_alignment = record.alignments[0]
        best_hsp = best_alignment.hsps[0]
        print(f"Mejor e-value: {best_hsp.expect}")
        print("Descripción del mejor hit:")
        print(best_alignment.hit_def)

    os.remove(query_fasta)

# Para usar en local, descomentar:
# blastn_local_desde_teclado()


## Ejercicio 2

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de proteína que introduzcas por teclado. El programa debe guardar en un fichero los resultados que tengan un E-value menor que 0.001, incluyendo: id, longitud, E-value y porcentaje de identidad. Hágalo de forma online y local.


### 2.a) BLASTP online con filtrado por E-value


In [ ]:
def blastp_online_filtrado(
    salida="blastp_online_filtrado.txt",
    evalue_umbral=0.001,
):
    from Bio.Blast import NCBIWWW, NCBIXML

    secuencia = input("Introduce la secuencia de proteína: ")
    secuencia = ''.join(secuencia.split()).upper()

    print("\nLanzando BLASTP online...")
    result_handle = NCBIWWW.qblast(
        program="blastp",
        database="nr",
        sequence=secuencia,
        expect=1.0,
        hitlist_size=100,
    )

    record = NCBIXML.read(result_handle)
    result_handle.close()

    with open(salida, "w") as f:
        f.write("id\tlongitud\tevalue\tporc_identidad\n")
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < evalue_umbral:
                    pid = porcentaje_identidad(hsp)
                    f.write(
                        f"{alignment.hit_id}\t{alignment.length}\t"
                        f"{hsp.expect}\t{pid:.2f}\n"
                    )

    print(f"Resultados guardados en {salida}")

# Para usar en local, descomentar:
# blastp_online_filtrado()



Lanzando BLASTP online...


ValueError: Error message from NCBI: Message ID#29 Error: Query string not found in the CGI context

### 2.b) BLASTP local con filtrado por E-value


In [50]:
def blastp_local_filtrado(
    db="proteinasbase",
    blastp_path="blastp",
    out_xml="blastp_local.xml",
    salida="blastp_local_filtrado.txt",
    evalue_umbral=0.001,
):
    from Bio.Blast import NCBIXML

    secuencia = input("Introduce la secuencia de proteína: ")
    secuencia = ''.join(secuencia.split()).upper()

    query_fasta = escribir_fasta_temporal(secuencia, descripcion="query_prot_local")

    blastp_cline = NcbiblastpCommandline(
        cmd=blastp_path,
        query=query_fasta,
        db=db,
        evalue=1.0,
        outfmt=5,
        out=out_xml,
    )

    print("Ejecutando BLASTP local...")
    stdout, stderr = blastp_cline()

    with open(out_xml) as handle:
        record = NCBIXML.read(handle)

    with open(salida, "w") as f:
        f.write("id\tlongitud\tevalue\tporc_identidad\n")
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < evalue_umbral:
                    pid = porcentaje_identidad(hsp)
                    f.write(
                        f"{alignment.hit_id}\t{alignment.length}\t"
                        f"{hsp.expect}\t{pid:.2f}\n"
                    )

    print(f"Resultados guardados en {salida}")
    os.remove(query_fasta)

# Para usar en local, descomentar:
# blastp_local_filtrado()


## Ejercicio 3

Escribe un programa que lea una secuencia de ADN desde un fichero FASTA, haga una búsqueda BLAST y filtre los resultados por organismo. Debe mostrar el número de resultados filtrados y el E-value medio. Hágalo online y local.


### 3.a) BLASTN online leyendo la query de un FASTA


In [51]:
def blastn_online_desde_fasta_filtrado_organismo(
    fasta_path,
    organismo,
    evalue_max=10.0,
):
    from Bio.Blast import NCBIWWW, NCBIXML

    record_fasta = SeqIO.read(fasta_path, "fasta")
    secuencia = str(record_fasta.seq)

    print(f"Query ID: {record_fasta.id}")
    print(f"Longitud: {len(secuencia)} bases")

    print("\nLanzando BLASTN online...")
    result_handle = NCBIWWW.qblast(
        program="blastn",
        database="nt",
        sequence=secuencia,
        expect=evalue_max,
        hitlist_size=100,
    )

    blast_record = NCBIXML.read(result_handle)
    result_handle.close()

    organismo_lower = organismo.lower()
    evalues_filtrados = []

    for alignment in blast_record.alignments:
        descripcion = alignment.hit_def.lower()
        if organismo_lower in descripcion:
            for hsp in alignment.hsps:
                evalues_filtrados.append(hsp.expect)

    num_resultados = len(evalues_filtrados)
    print(f"Número de resultados para '{organismo}': {num_resultados}")

    if num_resultados > 0:
        evalue_medio = statistics.mean(evalues_filtrados)
        print(f"E-value medio: {evalue_medio}")
    else:
        print("No hubo hits para ese organismo.")

# Ejemplo de uso en local:
# blastn_online_desde_fasta_filtrado_organismo("query.fasta", "Homo sapiens")


### 3.b) BLASTN local leyendo la query de un FASTA


In [52]:
def blastn_local_desde_fasta_filtrado_organismo(
    fasta_path,
    organismo,
    db="genomasbase",
    blastn_path="blastn",
    out_xml="blastn_local_fasta.xml",
    evalue_max=10.0,
):
    from Bio.Blast import NCBIXML

    blastn_cline = NcbiblastnCommandline(
        cmd=blastn_path,
        query=fasta_path,
        db=db,
        evalue=evalue_max,
        outfmt=5,
        out=out_xml,
    )

    print("Ejecutando BLASTN local desde FASTA...")
    stdout, stderr = blastn_cline()

    with open(out_xml) as handle:
        blast_record = NCBIXML.read(handle)

    organismo_lower = organismo.lower()
    evalues_filtrados = []

    for alignment in blast_record.alignments:
        descripcion = alignment.hit_def.lower()
        if organismo_lower in descripcion:
            for hsp in alignment.hsps:
                evalues_filtrados.append(hsp.expect)

    num_resultados = len(evalues_filtrados)
    print(f"Número de resultados para '{organismo}': {num_resultados}")

    if num_resultados > 0:
        evalue_medio = statistics.mean(evalues_filtrados)
        print(f"E-value medio: {evalue_medio}")
    else:
        print("No hubo hits para ese organismo.")

# Ejemplo en máquina con BLAST+:
# blastn_local_desde_fasta_filtrado_organismo("query.fasta", "Homo sapiens")


## Ejercicio 4

Lleve a cabo una búsqueda de ejemplo con cada una de las cinco utilidades de la suite descritas (`blastn`, `blastp`, `blastx`, `tblastn`, `tblastx`), tanto online como local. Comente qué especies tienen homólogos más cercanos y qué porcentaje de identidad y cobertura se observa.


Primero definimos una función para resumir el mejor hit de un BLAST (o los tres primeros).

In [53]:
def resumen_mejor_hit(blast_record, max_hits=3):
    if len(blast_record.alignments) == 0:
        print("No se obtuvieron hits.")
        return

    for i, alignment in enumerate(blast_record.alignments[:max_hits], start=1):
        best_hsp = alignment.hsps[0]
        pid = porcentaje_identidad(best_hsp)
        cov = cobertura_query(best_hsp, blast_record.query_length)
        print(f"Hit {i}:")
        print(f"  ID: {alignment.hit_id}")
        print(f"  Descripción: {alignment.hit_def}")
        print(f"  E-value: {best_hsp.expect}")
        print(f"  % identidad: {pid:.2f}")
        print(f"  Cobertura query: {cov:.2f}%")
        print()


### 4.a) BLAST online: blastn, blastp, blastx, tblastn, tblastx


In [54]:
from Bio.Blast import NCBIWWW, NCBIXML

def lanzar_blastn_online(secuencia_dna):
    result_handle = NCBIWWW.qblast("blastn", "nt", secuencia_dna)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

def lanzar_blastp_online(secuencia_prot):
    result_handle = NCBIWWW.qblast("blastp", "nr", secuencia_prot)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

def lanzar_blastx_online(secuencia_dna):
    result_handle = NCBIWWW.qblast("blastx", "nr", secuencia_dna)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

def lanzar_tblastn_online(secuencia_prot):
    result_handle = NCBIWWW.qblast("tblastn", "nt", secuencia_prot)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

def lanzar_tblastx_online(secuencia_dna):
    result_handle = NCBIWWW.qblast("tblastx", "nt", secuencia_dna)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

# Ejemplo de cómo los usaríamos en nuestra máquina:
# dna_ejemplo = "ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG"
# prot_ejemplo = "MKTIIALSYIFCLVFAD"
# rec_n = lanzar_blastn_online(dna_ejemplo); resumen_mejor_hit(rec_n)
# rec_p = lanzar_blastp_online(prot_ejemplo); resumen_mejor_hit(rec_p)
# rec_x = lanzar_blastx_online(dna_ejemplo); resumen_mejor_hit(rec_x)
# rec_tn = lanzar_tblastn_online(prot_ejemplo); resumen_mejor_hit(rec_tn)
# rec_tx = lanzar_tblastx_online(dna_ejemplo); resumen_mejor_hit(rec_tx)


### 4.b) BLAST local para las cinco utilidades


In [ ]:
from Bio.Blast import NCBIXML

def ejecutar_blast_local_y_resumir(cline, out_xml):
    print(cline)
    stdout, stderr = cline()
    with open(out_xml) as handle:
        blast_record = NCBIXML.read(handle)
    resumen_mejor_hit(blast_record)

def ejemplos_blast_local(dna_seq, prot_seq, db_nucl="genomasbase", db_prot="proteinasbase"):
    dna_seq = ''.join(dna_seq.split()).upper()
    prot_seq = ''.join(prot_seq.split()).upper()

    dna_fasta = escribir_fasta_temporal(dna_seq, "dna_query")
    prot_fasta = escribir_fasta_temporal(prot_seq, "prot_query")

    # 1) blastn
    blastn_xml = "blastn_local_ejemplo.xml"
    blastn_cline = NcbiblastnCommandline(query=dna_fasta, db=db_nucl, outfmt=5, out=blastn_xml)
    # ejecutar_blast_local_y_resumir(blastn_cline, blastn_xml)

    # 2) blastp
    blastp_xml = "blastp_local_ejemplo.xml"
    blastp_cline = NcbiblastpCommandline(query=prot_fasta, db=db_prot, outfmt=5, out=blastp_xml)
    # ejecutar_blast_local_y_resumir(blastp_cline, blastp_xml)

    # 3) blastx
    blastx_xml = "blastx_local_ejemplo.xml"
    blastx_cline = NcbiblastxCommandline(query=dna_fasta, db=db_prot, outfmt=5, out=blastx_xml)
    # ejecutar_blast_local_y_resumir(blastx_cline, blastx_xml)

    # 4) tblastn
    tblastn_xml = "tblastn_local_ejemplo.xml"
    tblastn_cline = NcbitblastnCommandline(query=prot_fasta, db=db_nucl, outfmt=5, out=tblastn_xml)
    # ejecutar_blast_local_y_resumir(tblastn_cline, tblastn_xml)

    # 5) tblastx
    tblastx_xml = "tblastx_local_ejemplo.xml"
    tblastx_cline = NcbitblastxCommandline(query=dna_fasta, db=db_nucl, outfmt=5, out=tblastx_xml)
    # ejecutar_blast_local_y_resumir(tblastx_cline, tblastx_xml)

    print("Se han preparado las llamadas para las cinco variantes BLAST en local.")
    print("Descomenta las líneas correspondientes para ejecutarlas en tu máquina.")

    os.remove(dna_fasta)
    os.remove(prot_fasta)

# Ejemplo de uso cuando tengas BLAST+:
# ejemplos_blast_local(dna_ejemplo, prot_ejemplo)


---

Con este cuaderno damos por resuelta la práctica de la Sesión 04 de laboratorio.
Hemos programado soluciones para las cuatro tareas: búsquedas BLAST online y locales, filtrado por E-value, filtrado por organismo y ejemplos con las cinco variantes de la suite BLAST.

Autores: Raúl Mendoza, Adrián Ojeda.